In [114]:
import numpy as np
from glob import glob 
import os
from garuda.ops import obb_iou

In [115]:
# old_directory = '/home/shataxi.dubey/shataxi_work/gujarat_brick_kilns/ahmedabad_10km_buffer_ground_truth_labels'
# new_directory = '/home/shataxi.dubey/shataxi_work/gujarat_brick_kilns/predict/ahmedabad_obb_v1__1120_300/ahmedabad_10km_buffer_images2/labels'

# old_directory = '/home/patel_zeel/kilns_neurips24/crossval/lucknow_sarath_grid_obb_v3/1/test/labels'
# new_directory = '/home/shataxi.dubey/shataxi_work/yolo_performance_on_empty_label_files/predict/test_lucknow_fold1_obb_v3__1120_100/lucknow_fold1/labels'

old_directory = '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed_data/multi_state_data_v1/labels' # ground truth
new_directory = '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels' # prediction

is_old_pred = False # is_old_pred = True when old directory contains predictions otherwise False
is_new_pred = False  # is_new_pred = True when new directory contains predictions otherwise False

old_files = glob(f'{old_directory}/*')
new_files = glob(f'{new_directory}/*')

In [116]:
np.loadtxt(old_files[0], ndmin = 2)

array([[0.      , 0.381183, 0.60867 , 0.403928, 0.5781  , 0.335328,
        0.527059, 0.312583, 0.55763 ],
       [0.      , 0.133058, 0.91044 , 0.154141, 0.876873, 0.086528,
        0.834406, 0.065445, 0.867973]])

In [117]:
def drop_extra_matches(matches: np.ndarray) -> np.ndarray: #copied from supervision
    """
    Deduplicate matches. If there are multiple matches for the same true or
    predicted box, only the one with the highest IoU is kept.
    """
    if matches.shape[0] > 0:
        matches = matches[matches[:, 2].argsort()[::-1]]
        matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
        matches = matches[matches[:, 2].argsort()[::-1]]
        matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
    return matches

In [118]:
files_not_present_previously = []
files_with_diff_num_boxes = []
files_with_incorrect_class = []
files_with_incorrect_box = []

box1 = 1 and box2 = 1
if half of the box2 intersects with box 1 then intersection is 0.5
IOU = 0.5/(1+1-0.5) = 0.33
So, it is better to keep IOU about 0.5 than 0.9

In [119]:
for new_file in new_files:
    new_fname = os.path.basename(new_file)
    old_file = os.path.join(old_directory,new_fname)
    # check if file exists
    if os.path.exists(old_file):
        new_labels = np.loadtxt(new_file,ndmin=2)
        old_labels = np.loadtxt(old_file,ndmin=2)
        # check if number of prediction matches
        if len(new_labels) == len(old_labels):
            if is_old_pred:
                old_boxes = old_labels[:,1:-1].astype(np.float32)
            else:
                old_boxes = old_labels[:,1:].astype(np.float32)
            if is_new_pred:
                new_boxes = new_labels[:,1:-1].astype(np.float32)
            else:
                new_boxes = new_labels[:,1:].astype(np.float32)
            # print('New box:', new_boxes, 'Old box:',old_boxes)
            iou = obb_iou(new_boxes, old_boxes)
            # print(iou)
            matched_idx = np.asarray(iou > 0.5).nonzero() 
            # print('matched_idx',matched_idx)
            matches = np.stack(
                (matched_idx[0], matched_idx[1], iou[matched_idx]), axis=1
            )
            matches = drop_extra_matches(matches=matches)
            # print(matches)
            if len(matches) != len(new_labels):
                files_with_incorrect_box.append(new_file)
            for box_match in matches:
                # check if class matches
                if new_labels[int(box_match[0])][0] != old_labels[int(box_match[1])][0]:
                #     print('Box matches')
                # else:
                    # print('Box does not match. Add it for hand validation')
                    files_with_incorrect_class.append(new_file)
        else:
            # print('Box does not match. Add it for hand validation')
            files_with_diff_num_boxes.append(new_file)
    else:
        # print('Box does not match. Add it for hand validation')
        files_not_present_previously.append(new_file)
    # break

In [120]:
files_with_incorrect_box

[]

In [121]:
files_with_incorrect_class

['/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/26.15,83.07.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/26.26,83.25.txt']

In [122]:
files_with_diff_num_boxes[:5]

['/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/26.13,82.79.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/26.23,83.26.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/25.99,83.12.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/25.72,83.16.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamga

In [123]:
len(files_with_incorrect_box)

0

In [124]:
len(files_with_diff_num_boxes)

20

In [125]:
len(files_not_present_previously)

317

In [126]:
handvalidate_files = files_with_incorrect_box + files_with_incorrect_class + files_with_diff_num_boxes + files_not_present_previously
print('Number of files to handvalidate: ',len(handvalidate_files))

Number of files to handvalidate:  339


In [127]:
handvalidate_files[:5]

['/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/26.15,83.07.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/26.26,83.25.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/26.13,82.79.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamgarh_images/labels/26.23,83.26.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/Heer/runs/obb/home/rishabh.mondal/Brick-Kilns-project/Heer/predict/train_up_ahmedabad_azamgarh_prediction_obb_v1__1120_100/azamga